# Proof of concept of ImViz requirements using glupyter/bqplot

We start off by silencing warnings that can happen when loading data as well as deprecation warnings, for clarity:

In [ ]:
import warnings
warnings.simplefilter('ignore')

Next we import all the required modules/classes/functions:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from bqplot import Scatter
from jdaviz import ImViz, ImVizTwoPanel
from glue.plugins.wcs_autolinking.wcs_autolinking import WCSLink

## Basic features

We start off by looking at some of the basic features using a single-image viewer jdaviz app:

In [ ]:
imviz = ImViz()
imviz.load_data('jw01072001001_01101_00001_nrcb1_cal.fits')

viewer = imviz.app.get_viewer('viewer-1')
viewer.add_data('jw01072001001_01101_00001_nrcb1_cal[SCI]')
viewer.state.show_axes = False
viewer.state.layers[0].percentile = 99

imviz.app

Panning and zooming is possible by showing the viewer toolbar and clicking on the '+'-shaped icon, then dragging around in the image and using scrolling to zoom in and out. To change the stretch and colormap, show the **Layer** options accessible through the last icon in the viewer toolbar.

We can also change these programmatically, for example the stretch:

In [ ]:
viewer.state.layers[0].stretch = 'sqrt'

the colormap:

In [ ]:
viewer.state.layers[0].cmap = plt.cm.viridis

the limits via the percentile option:

In [ ]:
viewer.state.layers[0].percentile = 90

or the limits directly:

In [ ]:
viewer.state.layers[0].v_min = -10
viewer.state.layers[0].v_max = +100

Note also that in the above example there are mouse-over coordinates visible by default.

## WCS Linking

Another ImViz requirement is the ability to show two images side by side and lock the field of view even if the WCSes are different. For now we have implemented a simple version of locking which is that every time the user clicks on a position, the field of view in both image viewers is updated to be centred on that position.

We set up a 2-panel version of ImViz and load in the two datasets:

In [ ]:
imviz = ImVizTwoPanel()
imviz.load_data('jw01072001001_01101_00001_nrcb1_cal.fits')
imviz.load_data('jw01072001001_01101_00005_nrcb1_cal.fits')

imviz.app.data_collection.add_link(WCSLink(imviz.app.data_collection['jw01072001001_01101_00001_nrcb1_cal[SCI]'],
                                           imviz.app.data_collection['jw01072001001_01101_00005_nrcb1_cal[SCI]'] ))

viewer1 = imviz.app.get_viewer('viewer-1')
viewer1.add_data('jw01072001001_01101_00001_nrcb1_cal[SCI]')
viewer1.state.show_axes = False
viewer1.state.layers[0].percentile = 99

viewer2 = imviz.app.get_viewer('viewer-2')
viewer2.add_data('jw01072001001_01101_00001_nrcb1_cal[SCI]')
viewer2.add_data('jw01072001001_01101_00005_nrcb1_cal[SCI]')
viewer2.state.show_axes = False
viewer2.state.layers[0].visible = False
viewer2.state.layers[1].percentile = 99
viewer2.state.reference_data = imviz.app.data_collection['jw01072001001_01101_00001_nrcb1_cal[SCI]']

imviz.app

Note that above we essentially add an invisible layer to the second image viewer so that we can select this dataset as a reference dataset in both image viewers, which means that even if the images have different WCS, the field of views will be identical.

If you click on the icon with the two rings (5th from the left in the toolbar) then click anywhere in the image in which you activated the tool the images will both be centred on that position.

We can also look at an example with very different resolutions and WCS:

In [ ]:
imviz = ImVizTwoPanel()
imviz.load_data('jw01072001001_01101_00001_nrcb1_cal.fits')
imviz.load_data('2mass_j.fits')

imviz.app.data_collection.add_link(WCSLink(imviz.app.data_collection['jw01072001001_01101_00001_nrcb1_cal[SCI]'],
                                           imviz.app.data_collection['2mass_j'] ))

viewer1 = imviz.app.get_viewer('viewer-1')
viewer1.add_data('jw01072001001_01101_00001_nrcb1_cal[SCI]')
viewer1.state.show_axes = False
viewer1.state.layers[0].percentile = 99

viewer2 = imviz.app.get_viewer('viewer-2')
viewer2.add_data('jw01072001001_01101_00001_nrcb1_cal[SCI]')
viewer2.add_data('2mass_j')
viewer2.state.show_axes = False
viewer2.state.layers[0].visible = False
viewer2.state.layers[1].percentile = 99
viewer2.state.reference_data = imviz.app.data_collection['jw01072001001_01101_00001_nrcb1_cal[SCI]']

imviz.app

## Overlaying data/transparency

A build-in feature in glue-jupyter is the ability to fade between layers:

In [ ]:
imviz = ImViz()
imviz.load_data('jw01072001001_01101_00001_nrcb1_cal.fits')
imviz.load_data('jw01072001001_01101_00005_nrcb1_cal.fits')

imviz.app.data_collection.add_link(WCSLink(imviz.app.data_collection['jw01072001001_01101_00001_nrcb1_cal[SCI]'],
                                           imviz.app.data_collection['jw01072001001_01101_00005_nrcb1_cal[SCI]'] ))

viewer1 = imviz.app.get_viewer('viewer-1')
viewer1.add_data('jw01072001001_01101_00001_nrcb1_cal[SCI]')
viewer1.add_data('jw01072001001_01101_00005_nrcb1_cal[SCI]')
viewer1.state.show_axes = False
viewer1.state.layers[0].percentile = 99
viewer1.state.layers[1].percentile = 99

imviz.app

If you go to the layer options and select the second layer in the drop-down and play with the opacity slider you can fade between the two images.

## Blinking

A variation on the above example is that we can set up a button (or in future a keyboard shortcut) to blink between images:

In [ ]:
imviz = ImViz()
imviz.load_data('jw01072001001_01101_00001_nrcb1_cal.fits')
imviz.load_data('jw01072001001_01101_00005_nrcb1_cal.fits')

imviz.app.data_collection.add_link(WCSLink(imviz.app.data_collection['jw01072001001_01101_00001_nrcb1_cal[SCI]'],
                                           imviz.app.data_collection['jw01072001001_01101_00005_nrcb1_cal[SCI]'] ))

viewer = imviz.app.get_viewer('viewer-1')
viewer.add_data('jw01072001001_01101_00001_nrcb1_cal[SCI]')
viewer.add_data('jw01072001001_01101_00005_nrcb1_cal[SCI]')
viewer.state.show_axes = False
viewer.state.layers[0].percentile = 99
viewer.state.layers[1].percentile = 99

imviz.app

If you press the 'b' key while your cursor is over the image, imviz will blink between the images.

## Astropy regions

It is already possible to make selections/regions in images and export these to astropy regions:

In [ ]:
imviz = ImViz()
imviz.load_data('jw01072001001_01101_00001_nrcb1_cal.fits')

viewer = imviz.app.get_viewer('viewer-1')
viewer.add_data('jw01072001001_01101_00001_nrcb1_cal[SCI]')
viewer.state.show_axes = False
viewer.state.layers[0].percentile = 99

imviz.app

Click on the viewer toolbar then click on the circular selection tool, and drag and click to select an interesting region on the sky. We can then export this region with:

In [ ]:
regions = imviz.app.get_subsets_from_viewer('viewer-1')

In [ ]:
regions

Since the region is an astropy region, we can e.g. convert it to a mask:

In [ ]:
mask = regions['Subset 1'].to_mask(mode='subpixels')

In [ ]:
plt.imshow(mask.to_image((2048, 2048)), origin='lower')

## DS9 mode

Do you want green circles wherever you click? If so, this section is for you!

In [ ]:
imviz = ImViz()
imviz.load_data('jw01072001001_01101_00001_nrcb1_cal.fits')

viewer = imviz.app.get_viewer('viewer-1')
viewer.add_data('jw01072001001_01101_00001_nrcb1_cal[SCI]')
viewer.state.show_axes = False
viewer.state.layers[0].percentile = 99

In [ ]:
scatter = Scatter(marker='circle', colors=['green'],
                  x=[], y=[],
                  scales={'x': viewer.figure.axes[0].scale,
                          'y': viewer.figure.axes[1].scale}, fill=False)
viewer.figure.marks = viewer.figure.marks + [scatter]

In [ ]:
def on_click(data):
    if data['event'] == 'click':
        x = data['domain']['x']
        y = data['domain']['y']
        scatter.x = np.hstack([scatter.x, x])
        scatter.y = np.hstack([scatter.y, y])

viewer.add_event_callback(on_click)

In [ ]:
imviz.app